In [ ]:
import re
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer  
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')

df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")

# Algemene Nederlandse stopwoorden
DUTCH_STOPWORDS = set(stopwords.words("dutch")) | {
    "bij", "voor", "met", "door", "zonder", "over",
    "doelgroep", "werk", "werken", "proces", "praktijk",
    "ontwikkeling", "ontwikkelen", "gaan",
    "leren", "school", "module", "thema",
    "student", "opleiding", "kun" , "vanuit",
    "eigen", "zelf", "samen", "samenwerken",
    "jaar", "week", "periode", "naasten", "daarnaast",
    "minor", "studenten", "programma", "keuzemodule",
    "casus", "casussen", "cases", "vraagstukken",
    "stage", "stageschool", "kennismakingsstage",
    "kennis", "vaardigheid", "vaardigheden", "ervaring", "ervaringen",
    "lessen", "onderwerpen", "theorie", "praktijk", "praktische", "inhoudelijke",
    "mee", "doe", "vinden", "vind", "kies", "openstaan",
    "belangrijk", "positief", "mogelijkheden","mogelijkheid",
    "impact", "betekenis", "betekent", "betekenen",
    "you", "your", "are", "will", "what", "then", "like", "choose",
    "interested", "experiencing", "hbo", "and", "the", "persoonlijke",
    "learning", "denken", "maken", "business", "verdieping", "emgeving",
    "bouwen", "thinking", "branding", "maken", "urban", "veiligheid",
    "nieuwe", "test", "gebouwde", "concept", "project", "omgeving", "actuele", "acute",
    "yellow", "belt", "serious", "hrm", "mensen", "snel", "binnen", "materialen", 
    "active", "druk", "context", "leven", "complexe" , "brede", "for", "jouw", "manieren"
}

def clean_text_for_matching(text: str) -> str:
    """Maak tekst schoon voor matching / uitleg."""
    text = str(text).lower()
    # haal alleen 'normale' woorden eruit
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    # filter stopwoorden en superkorte zooi
    tokens = [
        t for t in tokens
        if t not in DUTCH_STOPWORDS and len(t) > 2
    ]
    return " ".join(tokens)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeroe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 📘 Feedback van de docent

**Hieronder staat gerichte feedback op jullie notebook, gebaseerd op de rubric en opdracht.**

**Wat gaat goed:**
- Jullie hebben een functionele content‑based recommender.
- De code is netjes opgebouwd en duidelijk leesbaar.
- Basis NLP‑cleaning en similarity werkt goed.

**Wat ontbreekt / moet beter voor de rubric:**
- **Geen analyse van prestaties** (bijvoorbeeld uitleg waarom keuzes gemaakt zijn).
- **Geen optimalisatie/tuning** (bijv. parameters TF-IDF, embeddings testen).

**Aanbevolen verbeteringen:**
3. Voeg uitleg toe bij elke stap: waarom deze keuze, hoe werkt het model?
4. Voeg experimenten toe met TF‑IDF parameters, of evt. embeddings.
5. Toon eindresultaten in een tabel + korte motivatie waarom deze modules passen.

In [42]:
# Als je al combined_text hebt, gebruik die.
if "combined_text" in df.columns:
    base_text = df["combined_text"]
else:
    # Fallback: zelf combineren (pas aan naar wat jij wilt gebruiken)
    df["combined_text"] = (
        df["name"].fillna("") + " " +
        df["shortdescription"].fillna("") + " " +
        df["description"].fillna("") + " " +
        df["content"].fillna("") + " " +
        df["module_tags"].astype(str).fillna("") +" " +
        df["location"].astype(str).fillna("")
    )
    base_text = df["combined_text"]

# Schoongemaakte tekst voor recommender + uitleg
df["clean_text"] = base_text.apply(clean_text_for_matching)


In [43]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["clean_text"])

FEATURE_NAMES = vectorizer.get_feature_names_out()

In [44]:
def extract_match_terms(student_vec, module_vec, feature_names, max_terms: int = 8):
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])
    shared_idx = sorted(student_idx & module_idx)

    terms = [feature_names[i] for i in shared_idx]
    return terms[:max_terms]


In [45]:
import random

def _format_term_list(terms: list[str]) -> str:
    """Maak een nette NL opsomming: 'A', 'A en B', 'A, B en C'."""
    if not terms:
        return ""
    if len(terms) == 1:
        return f"'{terms[0]}'"
    if len(terms) == 2:
        return f"'{terms[0]}' en '{terms[1]}'"
    # 3 of meer
    quoted = [f"'{t}'" for t in terms]
    hoofd = ", ".join(quoted[:-1])
    laatste = quoted[-1]
    return f"{hoofd} en {laatste}"

def build_reason(match_terms: list[str], module_name: str | None = None, score: float | None = None) -> str:
    """
    Genereer een kort NL tekstje waarom de module past.
    - match_terms: termen waarop je matcht (interesses / keywords)
    - module_name: optioneel, naam van de module voor iets persoonlijkere tekst
    - score: optioneel, similarity-score (0–1) om bv. 'sterk' / 'goed' aan te sturen
    """
    terms_str = _format_term_list(match_terms)

    # score -> woordje voor sterkte (heel grof)
    if score is None:
        kwalificatie = "goed"
    elif score >= 0.8:
        kwalificatie = "erg goed"
    elif score >= 0.6:
        kwalificatie = "goed"
    else:
        kwalificatie = "redelijk"

    if not match_terms:
        # Geen specifieke termen: meer algemene tekst
        templates = [
            "Deze module sluit {kwalificatie} aan bij je interesses op basis van tekstuele overeenkomsten.",
            "Op basis van de overeenkomst tussen jouw profiel en de modulebeschrijving lijkt deze module {kwalificatie} bij je te passen.",
            "Deze module lijkt inhoudelijk {kwalificatie} aan te sluiten bij wat je interessant vindt."
        ]
    else:
        # Met match_terms
        if module_name:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}, zoals die terugkomen in '{module}'.",
                "Omdat {terms} centraal staan in '{module}', past deze module {kwalificatie} bij jouw interesses.",
                "Je interesse in {terms} komt duidelijk terug in '{module}', waardoor deze module {kwalificatie} bij je aansluit."
            ]
        else:
            templates = [
                "Deze module sluit {kwalificatie} aan bij je interesses in {terms}.",
                "Omdat {terms} in deze module aan bod komen, lijkt deze {kwalificatie} bij je te passen.",
                "Je interesse in {terms} komt terug in de inhoud van deze module, waardoor deze goed bij je past."
            ]

    template = random.choice(templates)
    return template.format(
        kwalificatie=kwalificatie,
        terms=terms_str,
        module=module_name if module_name else ""
    )


In [46]:
def recommend_modules( student_profile: str, top_n: int = 5, studycredit: int | None = None, level: str | None = None, location: str | None = None) -> pd.DataFrame:
    """
    Geeft top N modules voor een studentprofiel,
    inclusief similarity (genormaliseerd 0–1), match_terms en een NL-uitleg.
    """
    # Start met df + bijbehorende indices
    filtered_df = df.copy()

    # Filteren op studiepunten, niveau, locatie
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]
    if level is not None:
        filtered_df = filtered_df[filtered_df["level"].isin([level])]
    if location is not None:
        filtered_df = filtered_df[
            filtered_df["location"].str.contains(location, case=False, na=False)
        ]

    # Als er niks overblijft
    if filtered_df.empty:
        return filtered_df.assign(
            similarity=[],
            match_terms=[],
            reason=[]
        )

    # Studentprofiel vectoriseren
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    # Pak de juiste rijen uit de globale X aan de hand van de indexen van filtered_df
    row_indices = filtered_df.index.to_numpy()
    X_filtered = X[row_indices]

    # Cosine similarity
    sims = cosine_similarity(student_vec, X_filtered).flatten()

    # Ruwe similarity bewaren (optioneel, voor debug)
    filtered_df = filtered_df.copy()
    filtered_df["similarity_raw"] = sims

    # Normaliseren naar 0–1 binnen deze selectie
    max_sim = sims.max()
    if max_sim > 0:
        norm_sims = sims / max_sim
    else:
        norm_sims = sims  # alles 0

    filtered_df["similarity"] = norm_sims

    # Top N pakken
    top = filtered_df.sort_values("similarity", ascending=False).head(top_n)

    match_terms_list = []
    reasons = []

    for idx in top.index:
        # positie van deze rij binnen X_filtered
        pos = np.where(row_indices == idx)[0][0]
        module_vec = X_filtered[pos]

        terms = extract_match_terms(student_vec, module_vec, FEATURE_NAMES)
        match_terms_list.append(terms)

        reasons.append(
            build_reason(
                terms,
                module_name=top.at[idx, "name"],
                score=top.at[idx, "similarity"]  # genormaliseerde score
            )
        )

    top = top[["id", "name", "shortdescription", "similarity", "similarity_raw", "location", "studycredit", "level", "module_tags",]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top


In [47]:
sam_profile = """
Ik ben derdejaarsstudent en ik heb veel interesse in zorginnovatie,
data en AI. Ik wil graag modules volgen over zorgtechnologie, 
innovatie in de zorg en creatieve manieren om met technologie impact te maken.
"""

# sam_studycredit = 30
# sam_level = "NLQF6"
# sam_location = "Breda"


sam_studycredit = None
sam_level = None
sam_location = None

pd.set_option("display.max_colwidth", None)

# aanbevelingen = recommend_modules(sam_profile, top_n=5, studycredit=sam_studycredit, level=sam_level, location=sam_location)
aanbevelingen = recommend_modules(sam_profile, top_n=5, )
display(aanbevelingen)


,id,name,shortdescription,similarity,similarity_raw,location,studycredit,level,module_tags,match_terms,reason
17,176,Technologie die ècht werkt: innovatie in zorg en welzijn,"Technologie, smart health, robotica, zorg, welzijn",1.000000,0.205478,Breda,15,NLQF5,"['technologie', 'smart', 'health', 'robotica', 'zorg', 'welzijn']","[innovatie, technologie, zorg]","Je interesse in 'innovatie', 'technologie' en 'zorg' komt duidelijk terug in 'Technologie die ècht werkt: innovatie in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
13,172,Zorg dichtbij,"Zorg op afstand, technologie, implementeren, zorg, veranderkunde",0.986076,0.202617,Breda,15,NLQF5,"['zorg', 'afstand', 'technologie', 'implementeren', 'veranderkunde']","[technologie, zorg]","Deze module sluit erg goed aan bij je interesses in 'technologie' en 'zorg', zoals die terugkomen in 'Zorg dichtbij'."
140,331,Innovatiemanagement,"Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken. Het centrale thema van deze minor heeft betrekking op technologie en innovatie van nieuwe producten/diensten en processen binnen bestaande en nieuwe organisatie en de mogelijkheden van innovatie om verbeteringen in de bedrijfsvoering van organisaties door te voeren. Innovatie of vernieuwing is het invoeren van nieuwe ideeën, goederen, diensten en processen. Innovatie kan plaatsvinden binnen organisaties maar ook binnen bredere verbanden. Het proces van innovatie draait om dingen op een nieuwe (en zo mogelijk ook betere) manier aan te pakken.",0.858365,0.176375,Tilburg,30,NLQF6,"['innovatie draait', 'innovatie plaatsvinden', 'innovatie verbeteringen', 'innovatie', 'processen innovatie', 'innovatie vernieuwing', 'voeren innovatie', 'vernieuwing invoeren']","[innovatie, technologie]","Omdat 'innovatie' en 'technologie' centraal staan in 'Innovatiemanagement', past deze module erg goed bij jouw interesses."
26,185,Innoveren in zorg en welzijn,"Innoveren, zorgtechnologie, implementeren, ontwerpen, gezondheid",0.829365,0.170416,Breda,30,NLQF6,"['innoveren', 'zorgtechnologie', 'implementeren', 'ontwerpen', 'gezondheid']","[technologie, zorg, zorgtechnologie]","Je interesse in 'technologie', 'zorg' en 'zorgtechnologie' komt duidelijk terug in 'Innoveren in zorg en welzijn', waardoor deze module erg goed bij je aansluit."
64,237,BI Onderzoek voor innovatie,"onderzoek voor innovatie, prototype testing, implementatie onderzoek, concept test,",0.765155,0.157223,Den Bosch,15,NLQF5,"['onderzoek', 'innovatie', 'prototype', 'testing', 'implementatie']","[innovatie, manieren]","Omdat 'innovatie' en 'manieren' centraal staan in 'BI Onderzoek voor innovatie', past deze module goed bij jouw interesses."
